In [10]:
# from datasets import load_dataset
from fyp_work.paper_to_equation.generation.EquationGenerator import Equation
import sys


ModuleNotFoundError: No module named 'fyp_work'

In [9]:
print(sys.path)

['c:\\Users\\kyanj\\anaconda3\\envs\\fyp_env\\python311.zip', 'c:\\Users\\kyanj\\anaconda3\\envs\\fyp_env\\DLLs', 'c:\\Users\\kyanj\\anaconda3\\envs\\fyp_env\\Lib', 'c:\\Users\\kyanj\\anaconda3\\envs\\fyp_env', '', 'c:\\Users\\kyanj\\anaconda3\\envs\\fyp_env\\Lib\\site-packages', 'c:\\Users\\kyanj\\anaconda3\\envs\\fyp_env\\Lib\\site-packages\\win32', 'c:\\Users\\kyanj\\anaconda3\\envs\\fyp_env\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\kyanj\\anaconda3\\envs\\fyp_env\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\kyanj\\anaconda3\\envs\\fyp_env\\Lib\\site-packages\\setuptools\\_vendor']
